# Data Cleaning
We need to clean up these parquet files before we can use them for training the neural network. On a high level, we need to:
1. Figure out the linking strategy between impressions and conversions. I.e. which impressions lead to which conversions.
2. Ingest the data into a torch dataset.
   1. Remove unused or underused columns.
   2. Handle missing values
   3. Rename columns to be more descriptive.
   4. Parse any columns that need to be parsed.(eg. user-agent strings)
   5. Finally, think about what feature engineering needs to be done.


### Dataset file structure:
```
snapshot_20250429/
--- conversions/
------- imp_click_campaign_id=9317
-------------- dte=2025-01-01
--------------------- <numerous parquet files>
-------------- dte=2015-01-02
--------------------- <numerous parquet files>
-------------- ...
--------------------- <numerous parquet files>
-------------- dte=2025-04-14
--------------------- <numerous parquet files>
--- impressions/
------- campaign_id=9317
-------------- dte=2025-01-02
--------------------- <numerous parquet files>
-------------- dte=2025-01-03
--------------------- <numerous parquet files>
-------------- ...
-------------- dte=2025-04-14
--------------------- <numerous parquet files>
```


In [3]:
import pandas as pd

# Configure pandas display options for better readability
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000) # Adjust width for better table display if needed

impressions_path = './data/test_dataset/impressions_test/'
conversions_path = './data/test_dataset/conversions_test/'
device_types_path = './data/data_dictionary/device_types.csv'

# Load the impressions dataset
# Note: Parquet datasets can be stored as directories. Pandas reads them correctly.
df_impressions = pd.read_parquet(impressions_path)
df_conversions = pd.read_parquet(conversions_path)

# df_conversions[['imp_click_dttm_utc', 'conv_dttm_utc']].head()

df_impressions = df_impressions.drop(columns=[col for col in df_impressions.columns if col.startswith('aip')])
df_conversions = df_conversions.drop(columns=[col for col in df_conversions.columns if col.startswith('aip')])



## 1. Identify the linking strategy between impressions and conversions.
Claritas has updated the dataset to include a `unique_id` column on both the impressions and conversions datasets. This should make the linking strategy a lot easier.

DEPRECATED: I think the best way to do this is to look at the `imp_click_dttm_utc` and `conv_dttm_utc` columns in conjunction with the `campaign_id`, `placement_id`, and `imp_click_campaign_id`, `imp_click_placement_id` columns. I think that this should be unique, but I'm not sure at the moment.

In [4]:
import pandas as pd

# Assuming df_impressions and df_conversions are already loaded with the new data

# --- Linking using Unique IDs ---
# Perform a left join using the unique IDs
df_merged_final = pd.merge(
    df_impressions,
    df_conversions[['imp_click_unique_id', 'conv_dttm_utc', 'goal_id', 'goal_name']],
    left_on='unique_id',
    right_on='imp_click_unique_id',
    how='left'
)

# Create the conversion flag based on successful merge
# Check if a column from the right dataframe (conversions) is not null. 'conv_dttm_utc' is a good choice.
df_merged_final['conversion_flag'] = (~df_merged_final['conv_dttm_utc'].isnull()).astype(int)

# Optional: Drop the redundant ID column from the conversions table if desired
# df_merged_final = df_merged_final.drop(columns=['imp_click_unique_id'])

# --- Verification ---
print("--- Merged Data Info ---")
df_merged_final.info()

print(f"\nNumber of impressions successfully linked to a conversion: {df_merged_final['conversion_flag'].sum()}")

print("\n--- Sample of Merged Data ---")
# Display relevant columns to check the merge
display(df_merged_final[['unique_id', 'dttm_utc', 'conv_dttm_utc', 'goal_name', 'conversion_flag']].head())

# Check a few linked rows
print("\n--- Sample of Linked Rows (conversion_flag == 1) ---")
display(df_merged_final[df_merged_final['conversion_flag'] == 1][['unique_id', 'dttm_utc', 'conv_dttm_utc', 'goal_name', 'conversion_flag']].head())

--- Merged Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3845798 entries, 0 to 3845797
Data columns (total 17 columns):
 #   Column               Dtype         
---  ------               -----         
 0   placement_id         int64         
 1   dttm_utc             datetime64[ns]
 2   cnxn_type            object        
 3   user_agent           object        
 4   dma                  int32         
 5   country              object        
 6   os                   object        
 7   prizm_premier_code   object        
 8   device_type          object        
 9   unique_id            object        
 10  campaign_id          category      
 11  dte                  category      
 12  imp_click_unique_id  object        
 13  conv_dttm_utc        datetime64[ns]
 14  goal_id              float64       
 15  goal_name            object        
 16  conversion_flag      int64         
dtypes: category(2), datetime64[ns](2), float64(1), int32(1), int64(2), object(9)
m

,unique_id,dttm_utc,conv_dttm_utc,goal_name,conversion_flag
0,b0eea5e2-0f98-4609-b6c1-141118980e82,2025-01-02 14:49:33,NaT,NaN,0
1,b5352d12-7098-4378-929f-51db0bf40d8f,2025-01-02 18:48:43,NaT,NaN,0
2,75d06ce4-d3a2-4ec8-94ea-42c0ffaf6ed2,2025-01-02 11:14:21,NaT,NaN,0
3,b715b1c9-b150-4fbb-a5df-c02b4e252c9e,2025-01-02 03:12:29,NaT,NaN,0
4,43f8dc0d-3aee-4b7d-8b11-38bb9c778671,2025-01-02 18:13:09,NaT,NaN,0



--- Sample of Linked Rows (conversion_flag == 1) ---


,unique_id,dttm_utc,conv_dttm_utc,goal_name,conversion_flag
92,f9f72230-c174-40fe-b04d-a726297952d5,2025-01-02 22:38:19,2025-01-14 20:50:01,lead,1
93,f9f72230-c174-40fe-b04d-a726297952d5,2025-01-02 22:38:19,2025-01-14 15:09:29,lead,1
1248,164dc6fb-9af1-4bf0-8995-5947581c5e97,2025-01-02 20:42:42,2025-01-09 19:33:51,lead,1
1249,164dc6fb-9af1-4bf0-8995-5947581c5e97,2025-01-02 20:42:42,2025-01-09 19:32:10,lead,1
1250,164dc6fb-9af1-4bf0-8995-5947581c5e97,2025-01-02 20:42:42,2025-01-09 19:32:25,lead,1


## Step 2: Clean up the merged dataframe
We need to retain impression-level data for the most part in the merged dataframe since we're trying to predict the likelihood of a conversion given an impression.

In [6]:
# Select only the necessary columns from conversions for the merge
df_conversions_subset = df_conversions[['imp_click_unique_id', 'conv_dttm_utc', 'goal_name']].copy()

# Perform the left merge
df_merged_final = pd.merge(
    df_impressions,
    df_conversions_subset,
    left_on='unique_id',
    right_on='imp_click_unique_id',
    how='left'
)

print("Merge complete. Shape of merged DataFrame:", df_merged_final.shape)

Merge complete. Shape of merged DataFrame: (3845798, 15)


In [7]:
# Create the conversion flag: 1 if conv_dttm_utc is not NaT (Not a Time), 0 otherwise
df_merged_final['conversion_flag'] = (~df_merged_final['conv_dttm_utc'].isnull()).astype(int)

print("Conversion flag created.")
print("Number of conversions (conversion_flag == 1):", df_merged_final['conversion_flag'].sum())

Conversion flag created.
Number of conversions (conversion_flag == 1): 27863


In [8]:
# Drop the join key from the conversions table and other unnecessary conversion columns
# Note: We keep conv_dttm_utc and goal_name for now, will handle/drop later
columns_to_drop = ['imp_click_unique_id']
df_merged_final = df_merged_final.drop(columns=columns_to_drop)

print(f"Dropped columns: {columns_to_drop}")
print("Current columns:", df_merged_final.columns.tolist())

Dropped columns: ['imp_click_unique_id']
Current columns: ['placement_id', 'dttm_utc', 'cnxn_type', 'user_agent', 'dma', 'country', 'os', 'prizm_premier_code', 'device_type', 'unique_id', 'campaign_id', 'dte', 'conv_dttm_utc', 'goal_name', 'conversion_flag']


In [9]:
# Rename the impression timestamp column for clarity
df_merged_final = df_merged_final.rename(columns={'dttm_utc': 'impression_dttm_utc'})

print("Renamed 'dttm_utc' to 'impression_dttm_utc'.")

Renamed 'dttm_utc' to 'impression_dttm_utc'.


In [10]:
# Columns to check for missing values (impression features + goal_name)
cols_to_check_na = [
    'cnxn_type', 'user_agent', 'dma', 'country', 'os',
    'prizm_premier_code', 'device_type', 'goal_name'
]

# Calculate missing value percentages
missing_percentages = df_merged_final[cols_to_check_na].isnull().mean() * 100

print("--- Missing Value Percentages ---")
print(missing_percentages)

--- Missing Value Percentages ---
cnxn_type              0.000000
user_agent             0.000000
dma                    0.000000
country                0.000000
os                     0.000000
prizm_premier_code    63.149027
device_type           10.276021
goal_name             99.275495
dtype: float64


In [11]:
# Fill missing goal_name for non-conversions
df_merged_final['goal_name'] = df_merged_final['goal_name'].fillna('No Goal Name')

# Fill missing values in other categorical/object columns with 'Unknown'
# Adjust based on the analysis in 5a if needed (e.g., if 'user_agent' is 100% non-null)
cols_to_fill_unknown = ['prizm_premier_code', 'device_type']
for col in cols_to_fill_unknown:
    if col in df_merged_final.columns: # Check if column exists
         df_merged_final[col] = df_merged_final[col].fillna('Unknown')

# Verify remaining nulls (should ideally be 0 for these columns now)
print("\n--- Missing Values After Filling ---")
print(df_merged_final[cols_to_check_na].isnull().sum())


--- Missing Values After Filling ---
cnxn_type             0
user_agent            0
dma                   0
country               0
os                    0
prizm_premier_code    0
device_type           0
goal_name             0
dtype: int64


In [12]:
from user_agents import parse

# Apply the parser to the user_agent column
ua_features = df_merged_final['user_agent'].apply(lambda ua_str: pd.Series({
    'ua_browser': parse(ua_str).browser.family if ua_str else 'Unknown',
    'ua_os': parse(ua_str).os.family if ua_str else 'Unknown',
    'ua_device_family': parse(ua_str).device.family if ua_str else 'Unknown', # e.g., 'iPhone', 'Samsung SM-G950F'
    'ua_device_brand': parse(ua_str).device.brand if ua_str else 'Unknown', # e.g., 'Apple', 'Samsung'
    'ua_is_mobile': parse(ua_str).is_mobile,
    'ua_is_tablet': parse(ua_str).is_tablet,
    'ua_is_pc': parse(ua_str).is_pc,
    'ua_is_bot': parse(ua_str).is_bot,
}))

# Join the new features back to the main dataframe
df_merged_final = pd.concat([df_merged_final, ua_features], axis=1)

# Drop the original user_agent, os, and device_type columns as they are now redundant
columns_to_drop_after_ua = ['user_agent']
# Ensure columns exist before dropping
columns_to_drop_after_ua = [col for col in columns_to_drop_after_ua if col in df_merged_final.columns]
df_merged_final = df_merged_final.drop(columns=columns_to_drop_after_ua)


print("Parsed user_agent string and added new features.")
print("Dropped original 'user_agent', 'os', 'device_type' columns.")
print("New columns sample:", ua_features.head())

Parsed user_agent string and added new features.
Dropped original 'user_agent', 'os', 'device_type' columns.
New columns sample:                    ua_browser ua_os ua_device_family ua_device_brand  ua_is_mobile  ua_is_tablet  ua_is_pc  ua_is_bot
0  Mobile Safari UI/WKWebView   iOS           iPhone           Apple          True         False     False      False
1                    Podcasts   iOS       iOS-Device           Apple          True         False     False      False
2                    Podcasts   iOS       iOS-Device           Apple          True         False     False      False
3                    Podcasts   iOS       iOS-Device           Apple          True         False     False      False
4  Mobile Safari UI/WKWebView   iOS           iPhone           Apple          True         False     False      False


In [13]:
# Ensure the impression timestamp is in datetime format
df_merged_final['impression_dttm_utc'] = pd.to_datetime(df_merged_final['impression_dttm_utc'])

# Extract temporal features
df_merged_final['impression_hour'] = df_merged_final['impression_dttm_utc'].dt.hour
df_merged_final['impression_dayofweek'] = df_merged_final['impression_dttm_utc'].dt.dayofweek # Monday=0, Sunday=6

print("Added temporal features: 'impression_hour', 'impression_dayofweek'.")

Added temporal features: 'impression_hour', 'impression_dayofweek'.


In [15]:
print("\n--- Final DataFrame Info ---")
df_merged_final.info()

print("\n--- Final DataFrame Sample ---")
display(df_merged_final.head())

display(df_merged_final[df_merged_final["conversion_flag"] == 1].head())


--- Final DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3845798 entries, 0 to 3845797
Data columns (total 24 columns):
 #   Column                Dtype         
---  ------                -----         
 0   placement_id          int64         
 1   impression_dttm_utc   datetime64[ns]
 2   cnxn_type             object        
 3   dma                   int32         
 4   country               object        
 5   os                    object        
 6   prizm_premier_code    object        
 7   device_type           object        
 8   unique_id             object        
 9   campaign_id           category      
 10  dte                   category      
 11  conv_dttm_utc         datetime64[ns]
 12  goal_name             object        
 13  conversion_flag       int64         
 14  ua_browser            object        
 15  ua_os                 object        
 16  ua_device_family      object        
 17  ua_device_brand       object        
 18  ua_is_mobile

,placement_id,impression_dttm_utc,cnxn_type,dma,country,os,prizm_premier_code,device_type,unique_id,campaign_id,dte,conv_dttm_utc,goal_name,conversion_flag,ua_browser,ua_os,ua_device_family,ua_device_brand,ua_is_mobile,ua_is_tablet,ua_is_pc,ua_is_bot,impression_hour,impression_dayofweek
0,557650,2025-01-02 14:49:33,Corporate,602,us,iOS,Unknown,p,b0eea5e2-0f98-4609-b6c1-141118980e82,9317,2025-01-02,NaT,No Goal Name,0,Mobile Safari UI/WKWebView,iOS,iPhone,Apple,True,False,False,False,14,3
1,557650,2025-01-02 18:48:43,Cable/DSL,623,us,unknown,Unknown,p,b5352d12-7098-4378-929f-51db0bf40d8f,9317,2025-01-02,NaT,No Goal Name,0,Podcasts,iOS,iOS-Device,Apple,True,False,False,False,18,3
2,557650,2025-01-02 11:14:21,Cable/DSL,602,us,unknown,21,p,75d06ce4-d3a2-4ec8-94ea-42c0ffaf6ed2,9317,2025-01-02,NaT,No Goal Name,0,Podcasts,iOS,iOS-Device,Apple,True,False,False,False,11,3
3,557650,2025-01-02 03:12:29,Cable/DSL,602,us,unknown,21,p,b715b1c9-b150-4fbb-a5df-c02b4e252c9e,9317,2025-01-02,NaT,No Goal Name,0,Podcasts,iOS,iOS-Device,Apple,True,False,False,False,3,3
4,557650,2025-01-02 18:13:09,Cellular,602,us,iOS,Unknown,p,43f8dc0d-3aee-4b7d-8b11-38bb9c778671,9317,2025-01-02,NaT,No Goal Name,0,Mobile Safari UI/WKWebView,iOS,iPhone,Apple,True,False,False,False,18,3


,placement_id,impression_dttm_utc,cnxn_type,dma,country,os,prizm_premier_code,device_type,unique_id,campaign_id,dte,conv_dttm_utc,goal_name,conversion_flag,ua_browser,ua_os,ua_device_family,ua_device_brand,ua_is_mobile,ua_is_tablet,ua_is_pc,ua_is_bot,impression_hour,impression_dayofweek
92,557650,2025-01-02 22:38:19,Cable/DSL,602,us,iOS,Unknown,p,f9f72230-c174-40fe-b04d-a726297952d5,9317,2025-01-02,2025-01-14 20:50:01,lead,1,Spotify,iOS,iPhone,Apple,True,False,False,False,22,3
93,557650,2025-01-02 22:38:19,Cable/DSL,602,us,iOS,Unknown,p,f9f72230-c174-40fe-b04d-a726297952d5,9317,2025-01-02,2025-01-14 15:09:29,lead,1,Spotify,iOS,iPhone,Apple,True,False,False,False,22,3
1248,557650,2025-01-02 20:42:42,Cable/DSL,602,us,Android,Unknown,p,164dc6fb-9af1-4bf0-8995-5947581c5e97,9317,2025-01-02,2025-01-09 19:33:51,lead,1,Chrome Mobile,Android,K,Generic_Android,True,False,False,False,20,3
1249,557650,2025-01-02 20:42:42,Cable/DSL,602,us,Android,Unknown,p,164dc6fb-9af1-4bf0-8995-5947581c5e97,9317,2025-01-02,2025-01-09 19:32:10,lead,1,Chrome Mobile,Android,K,Generic_Android,True,False,False,False,20,3
1250,557650,2025-01-02 20:42:42,Cable/DSL,602,us,Android,Unknown,p,164dc6fb-9af1-4bf0-8995-5947581c5e97,9317,2025-01-02,2025-01-09 19:32:25,lead,1,Chrome Mobile,Android,K,Generic_Android,True,False,False,False,20,3
